In [0]:
# This notebook is meant to train a classification model from the Iris dataset and save it to the UC

In [0]:
%pip install mlflow --upgrade --pre
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 166.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-de9c4b31-9436-4be9-a101-60d217f02083
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Not uninstalling click at /databricks/python3/lib/python3.11/site-packages, outside e

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.tracking.client import MlflowClient
import requests

In [0]:
dbutils.widgets.text("catalog_name", "pedroz_e2edata_dev")
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
model_name = 'iris_model'

In [0]:
feature_table_name = f'{catalog_name}.default.iris_data'

In [0]:
experiment_name = f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/{model_name}_{catalog_name}"

In [0]:
# Create an MLFlow experiment
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3571252969788486', creation_time=1754402701666, experiment_id='3571252969788486', last_update_time=1754412791706, lifecycle_stage='active', name='/Users/pedro.zanlorensi@databricks.com/iris_model_pedroz_e2edata_dev', tags={'mlflow.databricks.filesystem.experiment_permissions_check': 'test',
 'mlflow.experiment.sourceName': '/Users/pedro.zanlorensi@databricks.com/iris_model_pedroz_e2edata_dev',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'pedro.zanlorensi@databricks.com',
 'mlflow.ownerId': '4529363229261380'}>

In [0]:
mlflow.autolog()

# Start a training run
with mlflow.start_run() as run:
    # Load data from Unity Catalog table
    df_iris = spark.table(feature_table_name).toPandas()
    features = ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm']
    target = 'species'

    X = df_iris[features]
    y = df_iris[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

    # Train the model
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and log metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    mlflow.log_metric("test_accuracy", accuracy)
    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_f1", f1)

    # Infer model signature
    signature = infer_signature(X_train, y_train)

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        name="model",
        signature=signature,
        input_example=X_train.head()
    )

    # Log input dataset for lineage
    data_source = mlflow.data.load_delta(table_name=feature_table_name)
    mlflow.log_input(data_source, context="training")

2025/08/08 16:57:39 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.1 <= scikit-learn <= 1.7.1, but the installed version is 1.3.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/08/08 16:57:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/08/08 16:57:40 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark: MLflow Spark dataset autologging is not supported on Databricks shared clusters or Databricks serverless clusters.
2025/08/08 16:57:40 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark.ml: [JVM_ATTRIBUTE_NOT_SUPPORTED] Attribute `sparkContext` is not supported in Spark Connect as it depends on the JVM. If you need to use this attribute, do not use Spark Connect when creating your session. Visit https://spark.apache.org/docs/latest/sq

In [0]:
# Out of all runs in the experiment, only register the run with the best selected metric
# Important note: this logic is optional and totally depends on your processes, so feel free to customize it!
# If you want, you can simply register the latest run instead, for example. 

selected_metric = 'test_accuracy'
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=[f"metrics.{selected_metric} DESC"], max_results=1)
best_run_id = runs[0].info.run_id

model_uri = f"runs:/{best_run_id}/model"
registered_model = mlflow.register_model(model_uri, f"{catalog_name}.default.{model_name}")
client.set_registered_model_alias(name=registered_model.name, alias="challenger", version=registered_model.version)

Registered model 'pedroz_e2edata_dev.default.iris_model' already exists. Creating a new version of this model...
2025/08/08 16:58:04 WARNING mlflow.tracking._model_registry.fluent: Run with id d6f62b4598754f0392c697d2f8e89e47 has no artifacts at artifact path 'model', registering model based on models:/m-f613615d6f3a424981824996f896a829 instead
🔗 Created version '20' of model 'pedroz_e2edata_dev.default.iris_model': https://adb-4181970831265458.18.azuredatabricks.net/explore/data/models/pedroz_e2edata_dev/default/iris_model/version/20?o=4181970831265458
